<a href="https://colab.research.google.com/github/riyakb/Relevant-Answer-Search-Engine/blob/master/preprocessing_qc_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#mounting drive

from google.colab import drive
drive.mount('/content/drive')


In [0]:
#changing path

import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Relevant Answer Search Engine")

In [0]:
#text_preprocessing

from nltk.corpus import wordnet
!pip install inflect
import inflect
import unicodedata
import re
import nltk
nltk.download('tagsets')
nltk.download('punkt')
from nltk.tag.perceptron import PerceptronTagger
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_words(words):
    """Lemmatize words in list of tokenized words"""
    lemmas=[]
    lemmatizer = WordNetLemmatizer()
    tagged = nltk.pos_tag(words)
    for word, tag in tagged:
        wntag = get_wordnet_pos(tag)
        if wntag is None:
            lemmas.append(lemmatizer.lemmatize(word))
        else:
            lemmas.append(lemmatizer.lemmatize(word, pos=wntag))
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words=lemmatize_words(words)#can use stem_words instead
    return words

def convert_string(string): #give any question/passage as input
    words = nltk.word_tokenize(string.lower())
    return normalize(words)

In [0]:
#NER

!pip install spacy==2.0.18
!python -m spacy download en_core_web_sm
!pip install msgpack==0.5.6

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

nlp = en_core_web_sm.load()
def ner(string): #give any question/passage as input
  doc = nlp(string)
  ner=[X.label_ for X in doc.ents]
  return ner


In [0]:
#question classifier + ner
import json

with open('qc.json') as F:
    q_type_dict = json.load(F)


def qc_ner(q,a):
  q_type=q_type_dict[q]
  a_ner=ner(a)
  if q_type==3:
    if 'PERSON' in a_ner:
      return 1
    else:
      return 0
  elif q_type==4:
      if ('NORP' in a_ner) or ('FAC' in a_ner) or ('ORG' in a_ner) or ('GPE' in a_ner) or ('LOC' in a_ner):
        return 1
      else:
        return 0
  elif q_type==5:
    if ('DATE' in a_ner) or ('TIME' in a_ner) or ('PERCENT' in a_ner) or ('MONEY' in a_ner) or ('QUANTITY' in a_ner) or ('ORDINAL' in a_ner) or ('CARDINAL' in a_ner):
      return 1
    else:
      return 0
  else:
    return 1

